In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
import sys

In [2]:
t1 = time.time()
df_train = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_all_count.csv')
# df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_fold_last_in_12_count.csv')
t2 = time.time()
print('training loading done! Time: {}'.format(t2 - t1))
print('size is: {}'.format(sys.getsizeof(df_train) / 1024 ** 3))
print('length is {}'.format(len(df_train)))

training loading done! Time: 211.47423267364502
size is: 16.07248181849718
length is 15408657


# Specify Categorical Columns

In [3]:
categorical_col = ['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'minute', 'second']
target = 'is_attributed'
feature_cols = list(set(df_train.columns) - set([target]))

# Get Validation

In [6]:
from sklearn.model_selection import train_test_split
trainset, valset = train_test_split(df_train,test_size=0.5, random_state=42)
print(sys.getsizeof(trainset)/ 1024 **3)

8.093642078340054


In [30]:
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

zeros = len(y_train[y_train == 0])
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train) * 100
import gc
# del df_train
gc.collect()

print(scale_pos_weight)

99.75660433979446


# Train Lightgbm

In [31]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 21,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 100,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.8, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':50,
        'bagging_fraction': 0.8, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': scale_pos_weight,
        'metric' : ['binary_logloss', 'auc']
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.961895	valid_0's binary_logloss: 0.236103
[20]	valid_0's auc: 0.965426	valid_0's binary_logloss: 0.116126
[30]	valid_0's auc: 0.965854	valid_0's binary_logloss: 0.0756788
[40]	valid_0's auc: 0.968169	valid_0's binary_logloss: 0.060358
[50]	valid_0's auc: 0.969177	valid_0's binary_logloss: 0.0530934
[60]	valid_0's auc: 0.970217	valid_0's binary_logloss: 0.0492494
[70]	valid_0's auc: 0.970638	valid_0's binary_logloss: 0.0469247
[80]	valid_0's auc: 0.970752	valid_0's binary_logloss: 0.0453847
[90]	valid_0's auc: 0.970766	valid_0's binary_logloss: 0.0441586
[100]	valid_0's auc: 0.970657	valid_0's binary_logloss: 0.0428453
[110]	valid_0's auc: 0.970608	valid_0's binary_logloss: 0.0421182
[120]	valid_0's auc: 0.9705	valid_0's binary_logloss: 0.0409858
[130]	valid_0's auc: 0.970489	valid_0's binary_logloss: 0.0402938
Early stopping, best iteration is:
[83]	valid_0's auc: 0.970813	valid_0's binary_logloss: 0.04

In [32]:
from sklearn.metrics import roc_auc_score
pred_val = model.predict(valset[feature_cols])
print(roc_auc_score(y_val,pred_val))

0.970813110142


In [33]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
importance.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/output/importance_train_count_all_0404_scale21.csv')

In [29]:
importance

ip                             1114
channel                         224
app                             152
second                          121
minute                          114
os                               76
hour                             48
ip_count                         43
ip_minute_count                  33
app_channel_count                18
ip_app_count                     16
ip_app_hour_count                15
ip_second_count                  15
device                           13
ip_app_os_count                  13
ip_day_hour_count                12
ip_os_hour_count                 12
ip_day_count                     10
ip_hour_count                    10
ip_device_hour_count             10
ip_day_minute_count               9
app_channel_day_count             9
device_os_count                   8
ip_os_day_count                   8
ip_device_minute_count            8
ip_device_second_count            8
ip_app_day_count                  8
app_second_count            

In [25]:
aa = model.predict(valset[feature_cols])

In [16]:
df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_fold_last_in_12_count.csv')


In [18]:
df_test = df_test[list(valset[feature_cols].columns)]
bb = model.predict(df_test)

In [19]:
list(set(valset[feature_cols].columns) - set(df_test.columns))

[]

In [20]:
bb

array([ 0.00341477,  0.00191355,  0.0016563 , ...,  0.9424411 ,
        0.00297018,  0.67816511])

In [33]:
list(valset[feature_cols].columns)

['ip_os_second_count',
 'app_day_second_count',
 'os_day_minute_count',
 'ip_channel_hour_count',
 'os_day_hour_count',
 'day_hour_count',
 'os_day_second_count',
 'app_count',
 'device_os_day_count',
 'day_second_count',
 'app_os_hour_count',
 'channel',
 'channel_minute_count',
 'ip_app_count',
 'day_count',
 'channel_day_count',
 'app_day_minute_count',
 'ip_minute_count',
 'app_hour_minute_count',
 'ip_os_count',
 'os_channel_count',
 'ip_device_minute_count',
 'os_count',
 'app_minute_count',
 'device_day_minute_count',
 'minute',
 'channel_day_second_count',
 'ip_device_hour_count',
 'ip_app_minute_count',
 'second',
 'hour',
 'device_os_count',
 'ip_device_day_count',
 'ip_os_minute_count',
 'ip_channel_day_count',
 'ip_device_count',
 'device_channel_count',
 'channel_second_count',
 'ip_os_channel_count',
 'ip_channel_minute_count',
 'app_second_count',
 'os_hour_minute_count',
 'minute_second_count',
 'ip_device_channel_count',
 'app_device_os_count',
 'channel_hour_second_co

In [18]:
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_fold_last_in_12csv.gz', compression='gzip', index=False)

In [21]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')

df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = bb

In [22]:
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_fold_6_in_6_last_count_0403_2_noscale.csv.gz', compression='gzip', index=False)

In [46]:
valset.head(1)

,ip,app,device,os,channel,day,hour,timestamp,minute,second,...,ip_app_device_count,ip_app_os_count,ip_app_channel_count,ip_device_os_count,ip_device_channel_count,ip_os_channel_count,app_device_os_count,app_device_channel_count,app_os_channel_count,device_os_channel_count
21638813,2348,15,1,13,245,8,23,1510182584,9,44,...,608.0,190.0,222.0,1963.0,313.0,96.0,2829042.0,4553892.0,1062250.0,1579048.0


In [45]:
df_test.head(1)

,ip,app,device,os,channel,day,hour,timestamp,minute,second,...,ip_app_device_count,ip_app_os_count,ip_app_channel_count,ip_device_os_count,ip_device_channel_count,ip_os_channel_count,app_device_os_count,app_device_channel_count,app_os_channel_count,device_os_channel_count
0,5744,9,1,3,107,10,4,1510286400,0,0,...,64.0,1.0,10.0,2.0,34.0,0.0,195701.0,504264.0,11217.0,174692.0
